In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split
# from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('../data/final_data/entrenamiento_postulaciones_vistas19_6_pp.csv', encoding='utf-8')
df.head()

,idaviso,idpostulante,se_postulo,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,month,day,dayofweek,sexo,edad
0,1112424009,96Xdv15,0,1,4,2,3,26,0,1,28
1,1112424009,96Xdv15,0,1,4,2,3,26,0,1,28
2,1112444846,8MaMbax,0,1,5,163,4,6,4,1,28
3,1112444846,8MaMbax,0,1,5,163,4,6,4,1,28
4,1112393575,VNx0axE,1,1,2,38,3,13,1,-1,36


In [3]:
df['se_postulo'] = df['se_postulo'].apply(lambda x: 1 if x >= 1 else 0)

In [4]:
cols = df.head(0)
cols.drop(['se_postulo', 'idpostulante', 'idaviso'],inplace=True, axis=1)
cols = cols.columns
cols

/home/prp-admin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Index(['idtipo_de_trabajo_int', 'idnivel_laboral_int', 'idnombre_area_int',
       'month', 'day', 'dayofweek', 'sexo', 'edad'],
      dtype='object')

In [5]:
dfix = df.loc[:,cols]
X = np.array(dfix)
y = np.array(df['se_postulo']) 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7506)

In [7]:
# clf1 = tree.DecisionTreeClassifier()
# clf2 = GradientBoostingClassifier()
# clf3 = VotingClassifier(estimators=[('tree', clf1), ('gb', clf2)])
# clf4 = GaussianNB()
# clf = VotingClassifier(estimators=[('gb', clf3), ('gnb', clf4)])
clf1 = make_pipeline(preprocessing.QuantileTransformer(), GaussianNB())
# clf2 = Perceptron()
clf2 = MLPClassifier()
clf = VotingClassifier(estimators=[('gnb', clf1), ('mlp', clf2)])
clf.fit(X_train, y_train)

VotingClassifier(estimators=[('gnb', Pipeline(memory=None,
     steps=[('quantiletransformer', QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,
          output_distribution='uniform', random_state=None,
          subsample=100000)), ('gaussiannb', GaussianNB(priors=None))])), ('mlp', ML...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [8]:
clf.score(X_test, y_test) # 0.822145383670987

/home/prp-admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8342160426911578

In [9]:
X_full_data = np.array(dfix)
pred_full_data = clf.predict(X_full_data)
df['sepostulo'] = pred_full_data
df.head()

/home/prp-admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,idaviso,idpostulante,se_postulo,idtipo_de_trabajo_int,idnivel_laboral_int,idnombre_area_int,month,day,dayofweek,sexo,edad,sepostulo
0,1112424009,96Xdv15,0,1,4,2,3,26,0,1,28,0
1,1112424009,96Xdv15,0,1,4,2,3,26,0,1,28,0
2,1112444846,8MaMbax,0,1,5,163,4,6,4,1,28,0
3,1112444846,8MaMbax,0,1,5,163,4,6,4,1,28,0
4,1112393575,VNx0axE,1,1,2,38,3,13,1,-1,36,1


## Subida 

In [10]:
top100_full = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
top100_full = pd.merge(top100_full, df[['idaviso','idpostulante', 'sepostulo']], how='left', on=['idaviso','idpostulante'])
top100_full.drop_duplicates(inplace=True)
top100_full = top100_full[['id', 'sepostulo']]
top100_full.fillna(0, inplace=True)
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full[top100_full['sepostulo'] == 1].shape

(60086, 2)

In [11]:
top100_full.shape

(100000, 2)

In [13]:
top100_full.to_csv('../data/entrega/Ensemble_GNB(scaler)_NNMLP.csv', encoding='utf-8', index=False)